### 获得日历

In [2]:
import datetime
import calendar

start_date = datetime.date(2013, 7, 1) # 开始日期
end_date = datetime.date(2017, 9, 30) # 结束日期

date_list = []
for year in range(start_date.year, end_date.year + 1):
    start_month = 1 if year > start_date.year else start_date.month
    end_month = 12 if year < end_date.year else end_date.month
    for month in range(start_month, end_month + 1):
        start_day = 1
        end_day = calendar.monthrange(year, month)[1]
        date_list.append((datetime.date(year, month, start_day), datetime.date(year, month, end_day)))

# 网站API限制，每次只能爬取一个月的数据，故需要每个月的开始时间和结束时间
start_date_ls = [] # 每个月的开始时间
end_date_ls = [] # 每个月的结束时间
# Print the list of start and end dates
for dates in date_list:
    # print(dates[0].strftime('%Y%m%d'), dates[1].strftime('%Y%m%d'))
    start_date_ls.append(str(dates[0].strftime('%Y%m%d')))
    end_date_ls.append(str(dates[1].strftime('%Y%m%d')))

if str(start_date.strftime('%Y%m%d')) < start_date_ls[0]: 
    start_date_ls[0] = start_date.strftime('%Y%m%d')
if str(end_date.strftime('%Y%m%d')) < end_date_ls[-1]: 
    end_date_ls[-1] = end_date.strftime('%Y%m%d')

print(len(start_date_ls))
print(len(end_date_ls))

for (a,b) in zip(start_date_ls,end_date_ls):
    print(a, b)

51
51
20130701 20130731
20130801 20130831
20130901 20130930
20131001 20131031
20131101 20131130
20131201 20131231
20140101 20140131
20140201 20140228
20140301 20140331
20140401 20140430
20140501 20140531
20140601 20140630
20140701 20140731
20140801 20140831
20140901 20140930
20141001 20141031
20141101 20141130
20141201 20141231
20150101 20150131
20150201 20150228
20150301 20150331
20150401 20150430
20150501 20150531
20150601 20150630
20150701 20150731
20150801 20150831
20150901 20150930
20151001 20151031
20151101 20151130
20151201 20151231
20160101 20160131
20160201 20160229
20160301 20160331
20160401 20160430
20160501 20160531
20160601 20160630
20160701 20160731
20160801 20160831
20160901 20160930
20161001 20161031
20161101 20161130
20161201 20161231
20170101 20170131
20170201 20170228
20170301 20170331
20170401 20170430
20170501 20170531
20170601 20170630
20170701 20170731
20170801 20170831
20170901 20170930


### 一、爬取数据，尚未处理
这段代码会运行十多分钟，所以运行前请检查一下数据是否爬下来了，以减少不必要的时间开销  
如果已经爬下来了，直接运行后面的代码就行
另外，每更换一个城市or地区，需要修改的代码有  
① city变量  
② url变量，这个需要在历史数据页面按F12，然后网络 → Fetch和XHR，找到一个historcial...的，点开即可发现api   
③ 上面的代码块，记得修改要收集的数据集的时间范围  
④ 修改时区，主要是下面代码块的两个lambda表达式  

In [3]:
import requests
from tqdm import tqdm

# Melbourne: {
# url: https://api.weather.com/v1/location/YMML:9:AU/observations/historical.json
# api_key: e1f10a1e78da46f5b10a1e78da96f525
# }
# LA: {
# url: https://api.weather.com/v1/location/KBUR:9:US/observations/historical.json
# api_key: e1f10a1e78da46f5b10a1e78da96f525
# }
# BAY(San Jose): {
# url: https://api.weather.com/v1/location/KSJC:9:US/observations/historical.json
# api_key: e1f10a1e78da46f5b10a1e78da96f525
# }
# Chicago: {
# url: https://api.weather.com/v1/location/KMDW:9:US/observations/historical.json
# api_key: e1f10a1e78da46f5b10a1e78da96f525
# }
# DC(Arlington, VA): {
# url: https://api.weather.com/v1/location/KSJC:9:US/observations/historical.json
# api_key: e1f10a1e78da46f5b10a1e78da96f525
# }
# New York City(NYC): {
# url: https://api.weather.com/v1/location/KLGA:9:US/observations/historical.json
# api_key: e1f10a1e78da46f5b10a1e78da96f525
# }


city = 'NYC'
url = "https://api.weather.com/v1/location/KLGA:9:US/observations/historical.json"
api_key = "e1f10a1e78da46f5b10a1e78da96f525"
my_list = []

# for循环爬取每个月的数据，加到my_list中，最后再concat到一起
for i in tqdm(range(len(start_date_ls))):
    params = {
        "apiKey": api_key,
        "units": "e",
        "startDate": start_date_ls[i],
        "endDate": end_date_ls[i]
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        data_list = data['observations']
        # data_list

        import pandas as pd
        data_DF_ori = pd.DataFrame(data_list)
        data_DF_ori

        from datetime import datetime
        import pytz

        data_DF_1 = data_DF_ori.copy()

        melbourne_timezone = pytz.timezone('Australia/Melbourne') # 收集不同城市数据集时，记得更改时区
        LA_timezone = pytz.timezone('America/Los_Angeles') 
        BAY_timezone = pytz.timezone('America/Los_Angeles') # 和LA同一个时区
        Chicago_timezone = pytz.timezone('America/Chicago')
        DC_timezone = pytz.timezone('US/Eastern')
        NYC_timezone = pytz.timezone('America/New_York')
        data_DF_1['valid_time_gmt'] = data_DF_ori['valid_time_gmt'].apply( \
            lambda x: datetime.fromtimestamp(x, tz=pytz.timezone('utc')).astimezone(NYC_timezone).replace(tzinfo=None))
        data_DF_1['expire_time_gmt'] = data_DF_ori['expire_time_gmt'].apply( \
            lambda x: datetime.fromtimestamp(x, tz=pytz.timezone('utc')).astimezone(NYC_timezone).replace(tzinfo=None))
        
        my_list.append(data_DF_1)

        # print(data)
    else:
        print(f"Error happens in iter {i} :", response.status_code, response.text)

data_DF = pd.concat(my_list, ignore_index=True)
data_DF


100%|██████████| 51/51 [02:10<00:00,  2.56s/it]


,key,class,expire_time_gmt,obs_id,obs_name,valid_time_gmt,day_ind,temp,wx_icon,icon_extd,...,clds,water_temp,primary_wave_period,primary_wave_height,primary_swell_period,primary_swell_height,primary_swell_direction,secondary_swell_period,secondary_swell_height,secondary_swell_direction
0,KLGA,observation,2013-07-01 01:51:00,KLGA,Nyc/La Guardia,2013-06-30 23:51:00,N,74.0,27.0,2700.0,...,BKN,None,None,None,None,None,None,None,None,None
1,KLGA,observation,2013-07-01 02:15:00,KLGA,Nyc/La Guardia,2013-07-01 00:15:00,N,73.0,26.0,2600.0,...,OVC,None,None,None,None,None,None,None,None,None
2,KLGA,observation,2013-07-01 02:51:00,KLGA,Nyc/La Guardia,2013-07-01 00:51:00,N,74.0,26.0,2600.0,...,OVC,None,None,None,None,None,None,None,None,None
3,KLGA,observation,2013-07-01 03:51:00,KLGA,Nyc/La Guardia,2013-07-01 01:51:00,N,74.0,26.0,2600.0,...,OVC,None,None,None,None,None,None,None,None,None
4,KLGA,observation,2013-07-01 04:44:00,KLGA,Nyc/La Guardia,2013-07-01 02:44:00,N,73.0,26.0,2600.0,...,OVC,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43426,KLGA,observation,2017-09-30 22:51:00,KLGA,Nyc/La Guardia,2017-09-30 20:51:00,N,56.0,27.0,2700.0,...,BKN,None,None,None,None,None,None,None,None,None
43427,KLGA,observation,2017-09-30 23:51:00,KLGA,Nyc/La Guardia,2017-09-30 21:51:00,N,56.0,27.0,2700.0,...,BKN,None,None,None,None,None,None,None,None,None
43428,KLGA,observation,2017-10-01 00:51:00,KLGA,Nyc/La Guardia,2017-09-30 22:51:00,N,55.0,27.0,2700.0,...,BKN,None,None,None,None,None,None,None,None,None
43429,KLGA,observation,2017-10-01 01:15:00,KLGA,Nyc/La Guardia,2017-09-30 23:15:00,N,55.0,27.0,2700.0,...,BKN,None,None,None,None,None,None,None,None,None


In [101]:
data_DF.to_csv(city+'Weather_ori_1.csv', index=False)

### 二、将爬取到的数据，将‘爬到的数据’与‘我们需要的’的重合部分，进行列名替换

In [102]:
"""
key: 观测站点的唯一标识符
class: 观测的类别
expire_time_gmt: 观测的过期时间
obs_id: 观测的ID
obs_name: 观测站点的名称
valid_time_gmt: 观测的有效时间
day_ind: 一天中的时间段（白天或晚上）
temp: 温度（华氏度）
wx_icon: 天气图标的代码
icon_extd: 天气图标的扩展代码
wx_phrase: 天气短语描述
pressure_tend: 气压变化趋势
pressure_desc: 气压描述
dewPt: 露点温度（华氏度）
heat_index: 酷热指数
rh: 相对湿度
pressure: 气压
vis: 能见度（英里）
wc: 风寒指数
wdir: 风向（角度）
wdir_cardinal: 风向的助记符号
gust: 风速瞬时值（英里/小时）
wspd: 风速（英里/小时）
max_temp: 最高温度（华氏度）
min_temp: 最低温度（华氏度）
precip_total: 总降水量（英寸）
precip_hrly: 每小时降水量（英寸）
snow_hrly: 每小时降雪量（英寸）
uv_desc: 紫外线指数描述
feels_like: 体感温度（华氏度）
uv_index: 紫外线指数
qualifier: 气象数据的限定条件
qualifier_svrty: 限定条件的严重程度
blunt_phrase: 概括的天气短语描述
terse_phrase: 简短的天气短语描述
clds: 云量
water_temp: 水温
primary_wave_period: 主波周期
primary_wave_height: 主波高度
primary_swell_period: 主涌浪周期
primary_swell_height: 主涌浪高度
primary_swell_direction: 主涌浪方向
secondary_swell_period: 次涌浪周期
secondary_swell_height: 次涌浪高度
secondary_swell_direction: 次涌浪方向"""

'\nkey: 观测站点的唯一标识符\nclass: 观测的类别\nexpire_time_gmt: 观测的过期时间\nobs_id: 观测的ID\nobs_name: 观测站点的名称\nvalid_time_gmt: 观测的有效时间\nday_ind: 一天中的时间段（白天或晚上）\ntemp: 温度（华氏度）\nwx_icon: 天气图标的代码\nicon_extd: 天气图标的扩展代码\nwx_phrase: 天气短语描述\npressure_tend: 气压变化趋势\npressure_desc: 气压描述\ndewPt: 露点温度（华氏度）\nheat_index: 酷热指数\nrh: 相对湿度\npressure: 气压\nvis: 能见度（英里）\nwc: 风寒指数\nwdir: 风向（角度）\nwdir_cardinal: 风向的助记符号\ngust: 风速瞬时值（英里/小时）\nwspd: 风速（英里/小时）\nmax_temp: 最高温度（华氏度）\nmin_temp: 最低温度（华氏度）\nprecip_total: 总降水量（英寸）\nprecip_hrly: 每小时降水量（英寸）\nsnow_hrly: 每小时降雪量（英寸）\nuv_desc: 紫外线指数描述\nfeels_like: 体感温度（华氏度）\nuv_index: 紫外线指数\nqualifier: 气象数据的限定条件\nqualifier_svrty: 限定条件的严重程度\nblunt_phrase: 概括的天气短语描述\nterse_phrase: 简短的天气短语描述\nclds: 云量\nwater_temp: 水温\nprimary_wave_period: 主波周期\nprimary_wave_height: 主波高度\nprimary_swell_period: 主涌浪周期\nprimary_swell_height: 主涌浪高度\nprimary_swell_direction: 主涌浪方向\nsecondary_swell_period: 次涌浪周期\nsecondary_swell_height: 次涌浪高度\nsecondary_swell_direction: 次涌浪方向'

In [103]:
import pandas as pd

# read the CSV file into a pandas dataframe
df = pd.read_csv(city+'Weather_ori_1.csv')

# create a dictionary to map the observation keys to the column names
key_to_col = {
    'key': None,
    'class': None,
    'expire_time_gmt': None,
    'obs_id': 'STATION',
    'obs_name': 'STATION_NAME',
    'valid_time_gmt': 'DATE',
    'day_ind': None,
    'temp': 'HOURLYDRYBULBTEMPF',
    'wx_icon': None,
    'icon_extd': None,
    'wx_phrase': None,
    'pressure_tend': 'HOURLYPressureTendency',
    'pressure_desc': 'HOURLYPressureChange',
    'dewPt': 'HOURLYDewPointTempF',
    'heat_index': None,
    'rh': 'HOURLYRelativeHumidity',
    'pressure': 'HOURLYStationPressure',
    'vis': 'HOURLYVISIBILITY',
    'wc': None,
    'wdir': 'HOURLYWindDirection',
    'wdir_cardinal': None,
    'gust': 'HOURLYWindGustSpeed',
    'wspd': 'HOURLYWindSpeed',
    'max_temp': 'DAILYMaximumDryBulbTemp',
    'min_temp': 'DAILYMinimumDryBulbTemp',
    'precip_total': 'DAILYPrecip',
    'precip_hrly': 'HOURLYPrecip',
    'snow_hrly': None,
    'uv_desc': None,
    'feels_like': None,
    'uv_index': None,
    'qualifier': None,
    'qualifier_svrty': None,
    'blunt_phrase': 'DAILYWeather',
    'terse_phrase': None,
    'clds': 'HOURLYSKYCONDITIONS',
    'water_temp': None,
    'primary_wave_period': None,
    'primary_wave_height': None,
    'primary_swell_period': None,
    'primary_swell_height': None,
    'primary_swell_direction': None,
    'secondary_swell_period': None,
    'secondary_swell_height': None,
    'secondary_swell_direction': None
}
key_to_col = {k: v for k, v in key_to_col.items() if v is not None}


# rename the columns using the dictionary
df = df.rename(columns=key_to_col)

# display the modified dataframe
# print(df.head())
df


,key,class,expire_time_gmt,STATION,STATION_NAME,DATE,day_ind,HOURLYDRYBULBTEMPF,wx_icon,icon_extd,...,HOURLYSKYCONDITIONS,water_temp,primary_wave_period,primary_wave_height,primary_swell_period,primary_swell_height,primary_swell_direction,secondary_swell_period,secondary_swell_height,secondary_swell_direction
0,KDCA,observation,2013-07-01 01:48:00,KDCA,Washington/Natl,2013-06-30 23:48:00,N,79.0,26.0,2600.0,...,OVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KDCA,observation,2013-07-01 01:52:00,KDCA,Washington/Natl,2013-06-30 23:52:00,N,79.0,26.0,2600.0,...,OVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KDCA,observation,2013-07-01 02:23:00,KDCA,Washington/Natl,2013-07-01 00:23:00,N,77.0,26.0,2600.0,...,OVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KDCA,observation,2013-07-01 02:52:00,KDCA,Washington/Natl,2013-07-01 00:52:00,N,76.0,26.0,2600.0,...,OVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KDCA,observation,2013-07-01 03:52:00,KDCA,Washington/Natl,2013-07-01 01:52:00,N,76.0,26.0,2600.0,...,OVC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47363,KDCA,observation,2017-09-30 21:52:00,KDCA,Washington/Natl,2017-09-30 19:52:00,N,62.0,33.0,3300.0,...,FEW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47364,KDCA,observation,2017-09-30 22:52:00,KDCA,Washington/Natl,2017-09-30 20:52:00,N,61.0,33.0,3300.0,...,FEW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47365,KDCA,observation,2017-09-30 23:52:00,KDCA,Washington/Natl,2017-09-30 21:52:00,N,60.0,33.0,3300.0,...,CLR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47366,KDCA,observation,2017-10-01 00:52:00,KDCA,Washington/Natl,2017-09-30 22:52:00,N,58.0,33.0,3300.0,...,CLR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
df.to_csv(city+'Weather_ori_2.csv', index=False)

### 三、只选择‘我们需要的’的列

In [105]:
import pandas as pd

# Read the original CSV file
df = pd.read_csv(city+'Weather_ori_2.csv')

selected_col = []
# Select the relevant columns
col = ['STATION', 'STATION_NAME', 'ELEVATION', 'LATITUDE', 'LONGITUDE', 'DATE', 'REPORTTYPE', 'HOURLYSKYCONDITIONS', 'HOURLYVISIBILITY', 'HOURLYPRSENTWEATHERTYPE', 'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC', 'HOURLYWETBULBTEMPF', 'HOURLYWETBULBTEMPC', 'HOURLYDewPointTempF', 'HOURLYDewPointTempC', 'HOURLYRelativeHumidity', 'HOURLYWindSpeed', 'HOURLYWindDirection', 'HOURLYWindGustSpeed', 'HOURLYStationPressure', 'HOURLYPressureTendency', 'HOURLYPressureChange', 'HOURLYSeaLevelPressure', 'HOURLYPrecip', 'HOURLYAltimeterSetting', 'DAILYMaximumDryBulbTemp', 'DAILYMinimumDryBulbTemp', 'DAILYAverageDryBulbTemp', 'DAILYDeptFromNormalAverageTemp', 'DAILYAverageRelativeHumidity', 'DAILYAverageDewPointTemp', 'DAILYAverageWetBulbTemp', 'DAILYHeatingDegreeDays', 'DAILYCoolingDegreeDays', 'DAILYSunrise', 'DAILYSunset', 'DAILYWeather', 'DAILYPrecip', 'DAILYSnowfall', 'DAILYSnowDepth', 'DAILYAverageStationPressure', 'DAILYAverageSeaLevelPressure', 'DAILYAverageWindSpeed', 'DAILYPeakWindSpeed', 'PeakWindDirection', 'DAILYSustainedWindSpeed', 'DAILYSustainedWindDirection', 'MonthlyMaximumTemp', 'MonthlyMinimumTemp', 'MonthlyMeanTemp', 'MonthlyAverageRH', 'MonthlyDewpointTemp', 'MonthlyWetBulbTemp', 'MonthlyAvgHeatingDegreeDays', 'MonthlyAvgCoolingDegreeDays', 'MonthlyStationPressure', 'MonthlySeaLevelPressure', 'MonthlyAverageWindSpeed', 'MonthlyTotalSnowfall', 'MonthlyDeptFromNormalMaximumTemp', 'MonthlyDeptFromNormalMinimumTemp', 'MonthlyDeptFromNormalAverageTemp', 'MonthlyDeptFromNormalPrecip', 'MonthlyTotalLiquidPrecip', 'MonthlyGreatestPrecip', 'MonthlyGreatestPrecipDate', 'MonthlyGreatestSnowfall', 'MonthlyGreatestSnowfallDate', 'MonthlyGreatestSnowDepth', 'MonthlyGreatestSnowDepthDate', 'MonthlyDaysWithGT90Temp', 'MonthlyDaysWithLT32Temp', 'MonthlyDaysWithGT32Temp', 'MonthlyDaysWithLT0Temp', 'MonthlyDaysWithGT001Precip', 'MonthlyDaysWithGT010Precip', 'MonthlyDaysWithGT1Snow', 'MonthlyMaxSeaLevelPressureValue', 'MonthlyMaxSeaLevelPressureDate', 'MonthlyMaxSeaLevelPressureTime', 'MonthlyMinSeaLevelPressureValue', 'MonthlyMinSeaLevelPressureDate', 'MonthlyMinSeaLevelPressureTime', 'MonthlyTotalHeatingDegreeDays', 'MonthlyTotalCoolingDegreeDays', 'MonthlyDeptFromNormalHeatingDD', 'MonthlyDeptFromNormalCoolingDD', 'MonthlyTotalSeasonToDateHeatingDD', 'MonthlyTotalSeasonToDateCoolingDD']


for col_name in col:
    if col_name in df.columns:
        selected_col.append(col_name)
print('the number of selected column is:', len(selected_col))
print(selected_col)

df = df[selected_col]

# Write
df.to_csv(city+'Weather_ori_3.csv', index=False)

the number of selected column is: 19
['STATION', 'STATION_NAME', 'DATE', 'HOURLYSKYCONDITIONS', 'HOURLYVISIBILITY', 'HOURLYDRYBULBTEMPF', 'HOURLYDewPointTempF', 'HOURLYRelativeHumidity', 'HOURLYWindSpeed', 'HOURLYWindDirection', 'HOURLYWindGustSpeed', 'HOURLYStationPressure', 'HOURLYPressureTendency', 'HOURLYPressureChange', 'HOURLYPrecip', 'DAILYMaximumDryBulbTemp', 'DAILYMinimumDryBulbTemp', 'DAILYWeather', 'DAILYPrecip']


### 四、手动加上经纬度，并对列重新进行整理排序，使得格式统一

In [106]:
# 新创建一个dataframe，然后遍历列表col（因为col中的列名是已经排序好了的），
# 即可完成整理排序，使得格式统一
new_df = pd.DataFrame()
for col_name in col:
    if col_name not in selected_col:
        new_df[col_name] = ''
    else:
        new_df[col_name] = df[col_name]

# 手动加上加经纬度
LATITUDE = 38.86 # 纬度
LONGITUDE = -77.06 # 经度
new_df['LATITUDE'] = LATITUDE
new_df['LONGITUDE'] = LONGITUDE

new_df.to_csv(city+'Weather_ori_4.csv', index=False)

# 以下无用，只是临时代码

In [22]:
import ast

s1 = "123"
s2 = "3.14"
s3 = "1.23e-4"
s4 = "abc"

# 使用 isdigit() 方法判断
print(s1.isdigit())  # 输出True
print(s2.isdigit())  # 输出False

# 使用 isdecimal() 方法判断
print(s1.isdecimal())  # 输出True
print(s2.isdecimal())  # 输出False

# 使用 isnumeric() 方法判断
print(s1.isnumeric())  # 输出True
print(s2.isnumeric())  # 输出True
print(s3.isnumeric())  # 输出True
print(s4.isnumeric())  # 输出False

# 使用 ast.literal_eval() 方法判断
try:
    a = ast.literal_eval(s1)
    print(type(a))  # 输出<class 'int'>
except:
    print(f"{s1} 不是数字")

try:
    b = ast.literal_eval(s2)
    print(type(b))  # 输出<class 'float'>
except:
    print(f"{s2} 不是数字")

try:
    c = ast.literal_eval(s3)
    print(type(c))  # 输出<class 'float'>
except:
    print(f"{s3} 不是数字")

try:
    d = ast.literal_eval(s4)
    print(type(d))
except:
    print(f"{s4} 不是数字")


True
False
True
False
True
False
False
False
<class 'int'>
<class 'float'>
<class 'float'>
abc 不是数字


In [10]:
import pandas as pd
import numpy as np

# 生成示例数据
data = pd.Series(np.random.randn(100))

# 找出异常值
mean = data.mean()
std = data.std()
threshold = mean + 2 * std
mask = data.abs() > threshold

# 将异常值所在的单元格涂为黄色
def color(val):
    if val in mask.index and mask.loc[val]:
        return 'background-color: yellow'
    else:
        return 'background-color: red'

# 创建DataFrame，并将异常值所在的单元格涂为黄色
df = pd.DataFrame({'data': data})
styled_df = df.style.applymap(color)
html = styled_df.render()  # 将样式表格渲染为HTML字符串
normal_df = pd.read_html(html)[0]  # 解析HTML字符串，获取第一个DataFrame对象
pd.ExcelWriter
normal_df.to_csv('output.csv', index=False)


In [32]:
data_list = data['observations']
data_list

[{'key': 'YMML',
  'class': 'observation',
  'expire_time_gmt': 1241107200,
  'obs_id': 'YMML',
  'obs_name': 'Melbourne/Tullam',
  'valid_time_gmt': 1241100000,
  'day_ind': 'N',
  'temp': 48,
  'wx_icon': 33,
  'icon_extd': 3300,
  'wx_phrase': 'Fair',
  'pressure_tend': None,
  'pressure_desc': None,
  'dewPt': 34,
  'heat_index': 48,
  'rh': 57,
  'pressure': 29.65,
  'vis': 6,
  'wc': 42,
  'wdir': 360,
  'wdir_cardinal': 'N',
  'gust': None,
  'wspd': 17,
  'max_temp': None,
  'min_temp': None,
  'precip_total': None,
  'precip_hrly': None,
  'snow_hrly': None,
  'uv_desc': 'Low',
  'feels_like': 42,
  'uv_index': 0,
  'qualifier': None,
  'qualifier_svrty': None,
  'blunt_phrase': None,
  'terse_phrase': None,
  'clds': 'CLR',
  'water_temp': None,
  'primary_wave_period': None,
  'primary_wave_height': None,
  'primary_swell_period': None,
  'primary_swell_height': None,
  'primary_swell_direction': None,
  'secondary_swell_period': None,
  'secondary_swell_height': None,
  'se

In [33]:
import pandas as pd
data_DF_ori = pd.DataFrame(data_list)
data_DF_ori


,key,class,expire_time_gmt,obs_id,obs_name,valid_time_gmt,day_ind,temp,wx_icon,icon_extd,...,clds,water_temp,primary_wave_period,primary_wave_height,primary_swell_period,primary_swell_height,primary_swell_direction,secondary_swell_period,secondary_swell_height,secondary_swell_direction
0,YMML,observation,1241107200,YMML,Melbourne/Tullam,1241100000,N,48,33,3300,...,CLR,None,None,None,None,None,None,None,None,None
1,YMML,observation,1241109000,YMML,Melbourne/Tullam,1241101800,N,48,33,3300,...,CLR,None,None,None,None,None,None,None,None,None
2,YMML,observation,1241112600,YMML,Melbourne/Tullam,1241105400,N,50,33,3390,...,CLR,None,None,None,None,None,None,None,None,None
3,YMML,observation,1241114400,YMML,Melbourne/Tullam,1241107200,N,50,33,3390,...,CLR,None,None,None,None,None,None,None,None,None
4,YMML,observation,1241116200,YMML,Melbourne/Tullam,1241109000,N,50,33,3300,...,CLR,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558,YMML,observation,1243776600,YMML,Melbourne/Tullam,1243769400,N,48,27,2700,...,BKN,None,None,None,None,None,None,None,None,None
1559,YMML,observation,1243778400,YMML,Melbourne/Tullam,1243771200,N,46,27,2700,...,BKN,None,None,None,None,None,None,None,None,None
1560,YMML,observation,1243780200,YMML,Melbourne/Tullam,1243773000,N,46,33,3300,...,CLR,None,None,None,None,None,None,None,None,None
1561,YMML,observation,1243782000,YMML,Melbourne/Tullam,1243774800,N,46,33,3300,...,CLR,None,None,None,None,None,None,None,None,None


In [34]:
from datetime import datetime
import pytz

data_DF_1 = data_DF_ori.copy()

melbourne_timezone = pytz.timezone('Australia/Melbourne')
data_DF_1['valid_time_gmt'] = data_DF_ori['valid_time_gmt'].apply( \
    lambda x: datetime.fromtimestamp(x, tz=pytz.timezone('utc')).astimezone(melbourne_timezone).replace(tzinfo=None))
data_DF_1['expire_time_gmt'] = data_DF_ori['expire_time_gmt'].apply( \
    lambda x: datetime.fromtimestamp(x, tz=pytz.timezone('utc')).astimezone(melbourne_timezone).replace(tzinfo=None))

data_DF_1

,key,class,expire_time_gmt,obs_id,obs_name,valid_time_gmt,day_ind,temp,wx_icon,icon_extd,...,clds,water_temp,primary_wave_period,primary_wave_height,primary_swell_period,primary_swell_height,primary_swell_direction,secondary_swell_period,secondary_swell_height,secondary_swell_direction
0,YMML,observation,2009-05-01 02:00:00,YMML,Melbourne/Tullam,2009-05-01 00:00:00,N,48,33,3300,...,CLR,None,None,None,None,None,None,None,None,None
1,YMML,observation,2009-05-01 02:30:00,YMML,Melbourne/Tullam,2009-05-01 00:30:00,N,48,33,3300,...,CLR,None,None,None,None,None,None,None,None,None
2,YMML,observation,2009-05-01 03:30:00,YMML,Melbourne/Tullam,2009-05-01 01:30:00,N,50,33,3390,...,CLR,None,None,None,None,None,None,None,None,None
3,YMML,observation,2009-05-01 04:00:00,YMML,Melbourne/Tullam,2009-05-01 02:00:00,N,50,33,3390,...,CLR,None,None,None,None,None,None,None,None,None
4,YMML,observation,2009-05-01 04:30:00,YMML,Melbourne/Tullam,2009-05-01 02:30:00,N,50,33,3300,...,CLR,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558,YMML,observation,2009-05-31 23:30:00,YMML,Melbourne/Tullam,2009-05-31 21:30:00,N,48,27,2700,...,BKN,None,None,None,None,None,None,None,None,None
1559,YMML,observation,2009-06-01 00:00:00,YMML,Melbourne/Tullam,2009-05-31 22:00:00,N,46,27,2700,...,BKN,None,None,None,None,None,None,None,None,None
1560,YMML,observation,2009-06-01 00:30:00,YMML,Melbourne/Tullam,2009-05-31 22:30:00,N,46,33,3300,...,CLR,None,None,None,None,None,None,None,None,None
1561,YMML,observation,2009-06-01 01:00:00,YMML,Melbourne/Tullam,2009-05-31 23:00:00,N,46,33,3300,...,CLR,None,None,None,None,None,None,None,None,None


In [36]:
data_DF_1.to_csv('Melbourne_tmp.csv', index=False)

In [8]:
import time
 
timeStamp = 1237986000
timeArray = time.localtime(timeStamp)
time_gmt = time.gmtime(timeStamp)
print(timeArray)
print(time_gmt)
otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
print(otherStyleTime)

time.struct_time(tm_year=2009, tm_mon=3, tm_mday=25, tm_hour=21, tm_min=0, tm_sec=0, tm_wday=2, tm_yday=84, tm_isdst=0)
time.struct_time(tm_year=2009, tm_mon=3, tm_mday=25, tm_hour=13, tm_min=0, tm_sec=0, tm_wday=2, tm_yday=84, tm_isdst=0)
2009-03-25 21:00:00


In [9]:
import pytz
from datetime import datetime

melbourne_tz = pytz.timezone('Australia/Melbourne')
unix_timestamp = 1237986000

utc_time = datetime.utcfromtimestamp(unix_timestamp)
melbourne_time = utc_time.replace(tzinfo=pytz.utc).astimezone(melbourne_tz)

print("Melbourne time is:", melbourne_time)


Melbourne time is: 2009-03-26 00:00:00+11:00


In [10]:
import csv

observations = {'observations': [{'key': 'YMML', 'class': 'observation', 'expire_time_gmt': 1237993200, 'obs_id': 'YMML', 'obs_name': 'Melbourne/Tullam', 'valid_time_gmt': 1237986000, 'day_ind': 'N', 'temp': 61, 'wx_icon': 33, 'icon_extd': 3300, 'wx_phrase': 'Fair', 'pressure_tend': None, 'pressure_desc': None, 'dewPt': 57, 'heat_index': 61, 'rh': 88, 'pressure': 29.56, 'vis': 6, 'wc': 61, 'wdir': 360, 'wdir_cardinal': 'N', 'gust': None, 'wspd': 8, 'max_temp': None, 'min_temp': None, 'precip_total': None, 'precip_hrly': None, 'snow_hrly': None, 'uv_desc': 'Low', 'feels_like': 61, 'uv_index': 0, 'qualifier': None, 'qualifier_svrty': None, 'blunt_phrase': None, 'terse_phrase': None, 'clds': 'FEW', 'water_temp': None, 'primary_wave_period': None, 'primary_wave_height': None, 'primary_swell_period': None, 'primary_swell_height': None, 'primary_swell_direction': None, 'secondary_swell_period': None, 'secondary_swell_height': None, 'secondary_swell_direction': None}]}

keys = observations['observations'][0].keys() # get the keys from the first item in the list

with open('observations.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, fieldnames=keys)
    dict_writer.writeheader()
    dict_writer.writerows(observations['observations'])
